# Self-follows on Kaggle
The follow feature on Kaggle was created by a Google engineer as a short [20% project](https://en.wikipedia.org/wiki/20%25_Project) as described [here](https://kaggle.com/general/33512).
A while ago I noticed that some users managed to follow themselves. I decided to take a closer look using the [Meta Kaggle](https://kaggle.com/kaggle/meta-kaggle) dataset.

In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

In [ ]:
import numpy as np
import pandas as pd
import plotly.express as px
from IPython.display import display, HTML

px.defaults.template = 'plotly_white'
px.defaults.color_discrete_sequence = ['steelblue']
MODE_BAR_BUTTONS = ['toImage', 'zoom2d', 'pan2d', 'select2d', 'lasso2d',
                    'zoomIn2d', 'zoomOut2d', 'autoScale2d', 'resetScale2d',
                    'toggleSpikelines', 'hoverClosestCartesian', 'hoverCompareCartesian'
                   ]
CONFIG = {
    'modeBarButtonsToRemove': ['pan2d', 'select2d', 'lasso2d', 'toggleSpikelines']
}

KAGGLE_BASE_URL = 'https://www.kaggle.com/'

novice = 'https://kaggle.com/static/images/avatier/avatier-novice@2x.png'
contributor = 'https://kaggle.com/static/images/avatier/avatier-contributor@2x.png'
expert = 'https://kaggle.com/static/images/avatier/avatier-expert@2x.png'
master = 'https://kaggle.com/static/images/avatier/avatier-master@2x.png'
grandmaster = 'https://kaggle.com/static/images/avatier/avatier-grandmaster@2x.png'
kaggle_staff = 'https://kaggle.com/static/images/avatier/avatier-staff@2x.png'

PERFORMANCE_TIERS = {0: 'Novice',
                     1: 'Contributor',
                     2: 'Expert',
                     3: 'Master',
                     4: 'Grandmaster',
                     5: 'Kaggle Team'
                    }

In [ ]:
users = pd.read_csv('/kaggle/input/meta-kaggle/Users.csv',
                    parse_dates=['RegisterDate'], 
                    infer_datetime_format=True
                   )
user_followers = pd.read_csv('/kaggle/input/meta-kaggle/UserFollowers.csv', 
                             parse_dates=['CreationDate'], 
                             infer_datetime_format=True
                            )

In [ ]:
self_follows = (user_followers[user_followers.UserId.eq(user_followers.FollowingUserId)]
                .merge(users, left_on='UserId', right_on='Id')
                .sort_values('CreationDate')
                .drop(columns=['Id_x', 'Id_y', 'UserId', 'FollowingUserId'])
                .rename(columns={'CreationDate': 'FollowDate'})
                .reset_index(drop=True)
               )
self_follows['PerfTier'] = self_follows['PerformanceTier']
self_follows['PerformanceTier'] = self_follows['PerformanceTier'].replace(PERFORMANCE_TIERS)

self_follows.loc[self_follows.PerfTier.eq(0), 'PerfTier'] = f'<img src="{novice}">'
self_follows.loc[self_follows.PerfTier.eq(1), 'PerfTier'] = f'<img src="{contributor}">'
self_follows.loc[self_follows.PerfTier.eq(2), 'PerfTier'] = f'<img src="{expert}">'
self_follows.loc[self_follows.PerfTier.eq(3), 'PerfTier'] = f'<img src="{master}">'
self_follows.loc[self_follows.PerfTier.eq(4), 'PerfTier'] = f'<img src="{grandmaster}">'
self_follows.loc[self_follows.PerfTier.eq(5), 'PerfTier'] = f'<img src="{kaggle_staff}">'

self_follows['User'] = '<a href="' + KAGGLE_BASE_URL + self_follows.UserName + '">' + self_follows.DisplayName + '</a>' + self_follows.PerfTier
self_follows['DisplayName'] = '<a href="' + KAGGLE_BASE_URL + self_follows.UserName + '">' + self_follows.DisplayName + '</a>'

columns = ['FollowDate', 'User', 'RegisterDate', 'PerformanceTier']
self_follows = self_follows.reindex(columns=columns)

self_follows.index = self_follows.index + 1

self_follow_string = f'There are currently {self_follows.shape[0]} users that follow themselves.'
display(HTML(self_follow_string))
display(HTML(self_follows.to_html(columns=['FollowDate', 'User', 'RegisterDate'],
                                  escape=False,
                                  render_links=True
                                 ).replace('<td>', '<td style="text-align:left">')))

In [ ]:
fig = px.scatter(self_follows,
                 x='FollowDate',
                 y=range(1, self_follows.shape[0] + 1)
                )
fig.update_traces(mode='lines',
                  hovertemplate=
                  '<b>%{x}</b><br>'+
                  'Number of self-follows: <b>%{y}</b>'
                 )
fig.update_layout(title='Number of self-follows over time',
                  xaxis_title='Date',
                  yaxis_title='Number of self-follows',
                  hoverlabel_bgcolor='white',
                  hoverlabel_font_size=14,
                  hovermode="x",
                  yaxis_zerolinecolor='grey',
                  yaxis_zerolinewidth=1
                 )
fig.show(config=CONFIG)

It looks like the first self-follow occurred on 30 December 2017. The second self-follow happened almost a year later. After the second self-follow, many other users begin following themselves.

In [ ]:
self_follows_by_tier = (self_follows
                        .PerformanceTier
                        .value_counts()
                        .to_frame()
                        .reset_index()
                        .rename(columns={'index': 'PerformanceTier', 'PerformanceTier': 'Count'})
                       )
fig = px.bar(self_follows_by_tier,
             x='Count',
             y='PerformanceTier',
             orientation='h'
            )
fig.update_traces(hovertemplate=
                  'Performance tier: <b>%{y}</b><br>'+
                  'Count: <b>%{x}</b><br>',
                  texttemplate='%{x}',
                  textposition='outside',
                  cliponaxis=False
                 )
fig.update_layout(title='Number of self-follows by performance tier',
                  xaxis_title='Number of self-follows',
                  yaxis_title='Performance tier',
                  yaxis_autorange='reversed',
                  hoverlabel_bgcolor="white",
                  hoverlabel_font_size=14,
                  hovermode="y"                  
                 )
fig.show(config=CONFIG)

I have not tried following myself but I found this [thead](https://kaggle.com/getting-started/126849) where a user describes being able to follow himself (but unable to revert this action) after finding his own notebook on the newsfeed. Since there is a "Follow" button next to users on the newsfeed, you could follow yourself if you happen to find your own work there.

Update (2020-08-31): With the [Profile Preview](https://www.kaggle.com/product-feedback/177372) update launched on Tuesday 25 August 2020, it is now much easier to follow yourself. Simply hover over your user avatar and click on "Follow" when the profile preview pops up.